## 0. Installing Dependencies

In [ ]:
# cd to SyferText/
# And run the below commands
# !conda activate openmined
# !git checkout psi_one_hot
# !pip uninstall syfertext -y
# !python setup.py install

In [ ]:
# # Install Git LFS
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
# !sudo apt-get install git-lfs
# !git lfs install

# # Install syfertext language model
# ! pip install git+git://github.com/Nilanshrajput/syfertext_en_core_web_lg@master

## Vocabulary in Private NLP

During training Language Models we often need to know the size of our vocabulary and represent the words in our vocabulary as one-hot-vectors. This is a trivial task while dealing with non-private and non-remote datasets. 

But things start to get complicated as we begin to deal with private datasets, cause the contents of the datasets are private we can't search for the unique words as before, thus making it a challenging task to represent the words as one-hot vectors.

Fret Not ! SyferText to the rescue !! ;)

In [6]:
import syfertext
import syft as sy
import sys
import torch
import random

from syft.generic.string import String

from syfertext.workers.virtual import VirtualWorker
from syfertext.encdec import encrypt, decrypt

hook = sy.TorchHook(torch)

In [2]:
# Remote workers
me = sy.local_worker
# me = syfertext.local_worker
# me = VirtualWorker(hook, "me")

# Language Object
nlp = syfertext.load("en_core_web_lg", owner=me)

bob = VirtualWorker(hook, "bob")
alice = VirtualWorker(hook, "alice")
carol = VirtualWorker(hook, "carol")
david = VirtualWorker(hook, "david")

In [3]:
# Simulate private datasets
workers = [bob, alice, carol, david]

texts = ["I am learning nlp during lockdown.",
         "I am learning cryptography during quarantine.",
         "I am learning more about privacy.",
         "I am building syfertext during quarantine.",
     ]

# Send text to workers
string_pointers = list()
for text, worker in zip(texts, workers):
    str_ptr = String(text).send(worker)
    string_pointers.append(str_ptr)

# Tokenize the strings and get doc pointers
docs = list()
for str_ptr in string_pointers:
    doc = nlp(str_ptr)
    docs.append(doc)

In [4]:
# Store the docs in corresponding variables
# Will be used later
bob_doc = docs[0]
alice_doc = docs[1]
carol_doc = docs[2]
david_doc = docs[3]

## 1. Private NLP Setup

While dealing with private datasets, we can't bring them to the local machine in their raw format. So instead we can first encrypt them on the remote machines and then bring them to the local machine for comparison. But, there are a few requirements for the encryption scheme that we should follow:

1. **Symmetric Key Encryption**: The keys used by each of the workers to encrypt their tokens should be same, cause only then the similar tokens reisdig on different workers can be compared to each other in their encrypted state.

2. **Determinstic Encryption**: The workers should encrypt their tokens using deterministic encryption, cause adding randomness to the encryption will not allow us to compare similar tokens even when they are encrypted with the same keys.

Hence, to fullfill these conditions we will be using the Diffie-Hellman Key exchange process and AES (Advance Encryption Scheme) in the ECB (Deterministic) Mode.

The PySyft BaseWorker has been extended with the following methods to allow us to perform the key exchange protocol easily.


```
_generate_private_key()
generate_public_key()
generate_secret_key()
```

In [5]:
shared_prime = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA18217C32905E462E36CE3BE39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9DE2BCBF6955817183995497CEA956AE515D2261898FA051015728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6BF12FFA06D98A0864D87602733EC86A64521F2B18177B200CBBE117577A615D6C770988C0BAD946E208E24FA074E5AB3143DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D788719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA993B4EA988D8FDDC186FFB7DC90A6C08F4DF435C93402849236C3FAB4D27C7026C1D4DCB2602646DEC9751E763DBA37BDF8FF9406AD9E530EE5DB382F413001AEB06A53ED9027D831179727B0865A8918DA3EDBEBCF9B14ED44CE6CBACED4BB1BDB7F1447E6CC254B332051512BD7AF426FB8F401378CD2BF5983CA01C64B92ECF032EA15D1721D03F482D7CE6E74FEF6D55E702F46980C82B5A84031900B1C9E59E7C97FBEC7E8F323A97A7E36CC88BE0F1D45B7FF585AC54BD407B22B4154AACC8F6D7EBF48E1D814CC5ED20F8037E0A79715EEF29BE32806A1D58BB7C5DA76F550AA3D8A1FBFF0EB19CCB1A313D55CDA56C9EC2EF29632387FE8D76E3C0468043E8F663F4860EE12BF2D5B0B7474D6E694F91E6DBE115974A3926F12FEE5E438777CB6A932DF8CD8BEC4D073B931BA3BC832B68D9DD300741FA7BF8AFC47ED2576F6936BA424663AAB639C5AE4F5683423B4742BF1C978238F16CBE39D652DE3FDB8BEFC848AD922222E04A4037C0713EB57A81A23F0C73473FC646CEA306B4BCBC8862F8385DDFA9D4B7FA2C087E879683303ED5BDD3A062B3CF5B3A278A66D2A13F83F44F82DDF310EE074AB6A364597E899A0255DC164F31CC50846851DF9AB48195DED7EA1B1D510BD7EE74D73FAF36BC31ECFA268359046F4EB879F924009438B481C6CD7889A002ED5EE382BC9190DA6FC026E479558E4475677E9AA9E3050E2765694DFC81F56E880B96E7160C980DD98EDD3DFFFFFFFFFFFFFFFFF
shared_base = 2

In [7]:
# Generate private keys
key_len = random.randint(0, 128) % 64
bob.generate_private_key(key_len)

key_len = random.randint(0, 128) % 64
alice.generate_private_key(key_len)

# Generate public keys
bob_public_key = bob.generate_public_key(shared_prime, shared_base)
alice_public_key = alice.generate_public_key(shared_prime, shared_base)

# Generate secret keys
alice.generate_secret_key(shared_prime, bob_public_key)
bob.generate_secret_key(shared_prime, alice_public_key)

Now Alice and Bob both have a **secret key**, which is same for both of them and known only to them. This enables us to get the encrypted tokens and find the distinct tokens among them.

In [8]:
bob_enc_tokens = bob_doc.get_encrypted_tokens_set()
alice_enc_tokens = alice_doc.get_encrypted_tokens_set()

# Print a sample of encrypted tokens
print("Bob's encrypted tokens:", bob_enc_tokens)
print("-"*20)
print("Alice's encrypted tokens:", alice_enc_tokens)

Bob's encrypted tokens: {b'10YvMHBVBFOecTtdhzWJJg==', b'Zx4vbNJ54aF8gkqNp3WBBA==', b'j/zIpbnjMag5438PC4vbgA==', b'XGk6Xzddijf4OYZj3nB/CQ==', b'GSIe1b+JgiXoVfY2gmBqeQ==', b'V0jknt6pDFJ5Tuzx3aiFQg==', b'8poeuCWT2bSkf9SP03Ov2A=='}
--------------------
Alice's encrypted tokens: {b'10YvMHBVBFOecTtdhzWJJg==', b'Zx4vbNJ54aF8gkqNp3WBBA==', b'mnX3g6mQL2w/+1D9VcWhkA==', b'gqS+mHl6dB0krvOj6SCHlA==', b'XGk6Xzddijf4OYZj3nB/CQ==', b'V0jknt6pDFJ5Tuzx3aiFQg==', b'8poeuCWT2bSkf9SP03Ov2A=='}


These sets can be compared to each other and the unique tokens can be determined from them.
Let's write some handy functions to do it.

In [13]:
import random

def assign_indices(all_tokens):
    """
    Args:
        all_tokens (Set): Set consisting
            tokens across all workers
    Returns:
        token_to_index (dict): Maps all unique 
            tokens to unique indices
    """
    all_tokens = list(all_tokens)
    
    # Shuffle the tokens to avoid any information
    # leakage due to the order of indices
    random.shuffle(all_tokens)
    
    indices = [i for i in range(len(all_tokens))]
    token_to_index = dict(zip(all_tokens, indices))    # Map tokens to indices
    return token_to_index

def map_to_indices(workers_tokens, token_to_index):
    """
    Args:
        workers_tokens (Iterable): Consists of tokens
            belonging to a specific worker.
        token_to_index (dict): Map of all unique tokens 
            to unique indices
    
    Returns:
        worker_token_to_index (dict): Each token in
            `worker_tokens` is mapped to an index
    """
    worker_token_to_index = {}    
    for token in workers_tokens:
        # map token to index
        worker_token_to_index[token] = token_to_index[token]    
    return worker_token_to_index

In [14]:
def print_dict(token_to_index):
    for enc_token, index in token_to_index.items():
        print("Encrypted Token: ", enc_token, "\t\tIndex: ", index)


all_tokens = bob_enc_tokens.union(alice_enc_tokens)  # Take the union of both sets
VOCAB_SIZE = len(all_tokens)                         # store the vocaulary size

token_to_indices = assign_indices(all_tokens)

print("Vocabulary size: ", VOCAB_SIZE, "\n\n")
print_dict(token_to_indices)

Vocabulary size:  9 


Encrypted Token:  b'10YvMHBVBFOecTtdhzWJJg==' 		Index:  0
Encrypted Token:  b'GSIe1b+JgiXoVfY2gmBqeQ==' 		Index:  1
Encrypted Token:  b'j/zIpbnjMag5438PC4vbgA==' 		Index:  2
Encrypted Token:  b'Zx4vbNJ54aF8gkqNp3WBBA==' 		Index:  3
Encrypted Token:  b'V0jknt6pDFJ5Tuzx3aiFQg==' 		Index:  4
Encrypted Token:  b'8poeuCWT2bSkf9SP03Ov2A==' 		Index:  5
Encrypted Token:  b'XGk6Xzddijf4OYZj3nB/CQ==' 		Index:  6
Encrypted Token:  b'mnX3g6mQL2w/+1D9VcWhkA==' 		Index:  7
Encrypted Token:  b'gqS+mHl6dB0krvOj6SCHlA==' 		Index:  8


In [15]:
# Assign indices to bob's tokens
bob_token_to_index = map_to_indices(bob_enc_tokens, token_to_indices)

# Assign indices to alice's tokens
alice_token_to_index = map_to_indices(alice_enc_tokens, token_to_indices)

print("Bob's Tokens\n")
print_dict(bob_token_to_index)

print("\nAlice's Tokens\n")
print_dict(alice_token_to_index)

Bob's Tokens

Encrypted Token:  b'10YvMHBVBFOecTtdhzWJJg==' 		Index:  0
Encrypted Token:  b'Zx4vbNJ54aF8gkqNp3WBBA==' 		Index:  3
Encrypted Token:  b'j/zIpbnjMag5438PC4vbgA==' 		Index:  2
Encrypted Token:  b'XGk6Xzddijf4OYZj3nB/CQ==' 		Index:  6
Encrypted Token:  b'GSIe1b+JgiXoVfY2gmBqeQ==' 		Index:  1
Encrypted Token:  b'V0jknt6pDFJ5Tuzx3aiFQg==' 		Index:  4
Encrypted Token:  b'8poeuCWT2bSkf9SP03Ov2A==' 		Index:  5

Alice's Tokens

Encrypted Token:  b'10YvMHBVBFOecTtdhzWJJg==' 		Index:  0
Encrypted Token:  b'Zx4vbNJ54aF8gkqNp3WBBA==' 		Index:  3
Encrypted Token:  b'mnX3g6mQL2w/+1D9VcWhkA==' 		Index:  7
Encrypted Token:  b'gqS+mHl6dB0krvOj6SCHlA==' 		Index:  8
Encrypted Token:  b'XGk6Xzddijf4OYZj3nB/CQ==' 		Index:  6
Encrypted Token:  b'V0jknt6pDFJ5Tuzx3aiFQg==' 		Index:  4
Encrypted Token:  b'8poeuCWT2bSkf9SP03Ov2A==' 		Index:  5


In [16]:
# Let's now return the token_to_index to the respective docs
# So that later on we can query them to get index for the tokens
# they contain.
bob_doc.set_indices(bob_token_to_index)
alice_doc.set_indices(alice_token_to_index)

So, we are able to find all the unique words across all workers without leaking any information about:
1. Tokens that the Bob and Alice contain to the local worker
2. Tokens that Bob contains to Alice and vice-versa

But, if there is a huge limitation to this approach. Let's see what it is.

## 2. Man-In-The-Middle Attack

One of the major drawbacks with the **public keys passing through the local worker is that the local worker can perform the man-in-the-middle attack**, which is a well-known limitation of the diffie-hellman key exchange. This allows the local worker to decrypt both Alice and Bob's tokens.

Note: `The key concept of this attack is that the local worker impersonates itself as a data provider. And it establishes trus with both Bob and Alice thus enabling it to decrypt both of their tokens.`.

Let's demonstrate how this can be done. 

In [18]:
# Local worker capable of performing DH key exchange
local_worker = VirtualWorker(hook, "local_worker")

# Generate private key for local worker
key_len = random.randint(0, 128) % 64
local_worker.generate_private_key(key_len)

# Generate public key for local worker
local_public_key = local_worker.generate_public_key(shared_prime, shared_base)

In [19]:
# Generate fresh public keys for bob and alice
bob_public_key = bob.generate_public_key(shared_prime, shared_base)
alice_public_key = alice.generate_public_key(shared_prime, shared_base)

In [20]:
""" This is the Key Step. """
# Now instead of sending bob's public key to alice and alice's 
# public key to bob, the local worker can pass both of them it's
# own public key.
alice.generate_secret_key(shared_prime, local_public_key)
bob.generate_secret_key(shared_prime, local_public_key)

# And using bob's public key, local worker
# can generate a secret key which is similar to the one
# with bob.
local_worker.generate_secret_key(shared_prime, bob_public_key)
bob_compromised_key = local_worker.secret_key

# And using alice's public key, local worker
# can generate a secret key which is similar to the one
# with alice.
local_worker.generate_secret_key(shared_prime, alice_public_key)
alice_compromised_key = local_worker.secret_key

print("Bob's compromised secret key   : \n", bob_compromised_key, '\n')
print("Alice's compromised secret key : \n", alice_compromised_key)

Bob's compromised secret key   : 
 b',\xc4\xe1\x9d\x14o\x037\x1f\xf5\x08E\xd3\xee\xde.\x12\xc1\xc5\xae\x83G\x93\xcc\x9b\xdf\xd2\x01C\xb7v\x00' 

Alice's compromised secret key : 
 b'\x9f\x8dG\xb2\x9e\xf9Q\xe0%yc\xb67acBOV\x0c\x8f\x91\x8e\xdca\xe79I\xc5N\\\xe3\x99'


Now let us try to bob's and alice's tokens using their compromised secret keys.

In [21]:
# First we will get the encrypted tokens
bob_enc_tokens = bob_doc.get_encrypted_tokens_set()
alice_enc_tokens = alice_doc.get_encrypted_tokens_set()

# Print the encrypted tokens tokens
print("Bob's data:", bob_enc_tokens)
print('-'*10)
print("Alice's data:", alice_enc_tokens)

Bob's data: {b'0B7YpBgrKjM2WGpkl12O+A==', b'3LcjNspRlarwehR9tUui8Q==', b'9C8qeM4SmAaCYJ2I8R/i9w==', b'sXv3DNDX8zpBB9Dgd4DxMA==', b'NmueVuCywxR2xFF8vH67XQ==', b'wFDm2nG3GaU9UVh4mPibKw==', b'Wd+1O+CFV9qKCHw3b/aLFA=='}
----------
Alice's data: {b'nqwiXu7aQGOKdYVNqOU0Eg==', b'qDIlpOXHnlJHo+rlWMyIbg==', b'ecc44PQn47cZNmRzt5NYuw==', b'L/h44wQ/zSSZiCI4eTJMsA==', b'JRrd4g7P3kvZtTWN+CtFng==', b'MqCnbKCiGDQg1fmqZ5U2Ow==', b'+35fRy0xRRo3Pb0M8+GzUg=='}


In [22]:
def decrypt_tokens(enc_tokens, secret_key):
    """ Decrypts a set of enc_tokens using
    passsed in secret_key. """
    dec_tokens = list()
    for token in enc_tokens:
        dec_token = decrypt(token, secret_key).decode("utf-8")
        dec_tokens.append(dec_token)
    return dec_tokens

bob_dec_tokens = decrypt_tokens(bob_enc_tokens, 
                                bob_compromised_key)

alice_dec_tokens = decrypt_tokens(alice_enc_tokens, 
                                  alice_compromised_key)

# Print the decrypted tokens
print("Bob's private data:", bob_dec_tokens)
print('-'*10)
print("Alice's private data:", alice_dec_tokens)

Bob's private data: ['nlp', 'learning', 'am', 'I', 'during', 'lockdown', '.']
----------
Alice's private data: ['learning', 'during', 'cryptography', 'quarantine', 'I', '.', 'am']


In just a few extra steps the local worker is able to decrypt both Bob's and Alice's encrypted tokens. So we must find a solution to avoid this.

## 3. DHKE on SecureWorker

DHKE: Diffie-Hellman Key Exchange

In general, the local worker is the one who is training a model.
For eg, it can be a company developing a health product and training a Deep Learning model on the data residing on the hospitals servers.
But as mentioned above, if the local worker is malicious, it is capable of decrypting the data present on the hospital's machines.

Thus, in order to avoid the data-breach and protect the privacy of the data belonging to the workers, the **workers can decide to trust a third-party worker** who acts as a neutral party. This worker called as the SecurWorker is trustworthy and **performs the Diffie-Hellman Key Exchange** between all workers.

So, in our hospital example, the SecureWorker can be an **AWS instance**. Moreover, it is required only for a very short instance.

The SecureWorker is capable of performing a secure key-exchange process between two or more workers. Here we will present an example, with **four workers**.

In [23]:
workers = [bob, alice, carol, david]
secure_worker = VirtualWorker(hook, id="james")

# Execute the DH key exchange protocol securely on Secure Worker
secure_worker.execute_dh_key_exchange(shared_prime, shared_base, workers)

##### And that's it!! ;)

Now all the workers have a secret key on their machines, which is same for all of them, and thus enables us the local-worker to perform the **Set Intersection**.

In [24]:
# List of Sets of encryted tokens
enc_token_sets = list()

for doc in docs:
    enc_tokens = doc.get_encrypted_tokens_set()
    enc_token_sets.append(enc_tokens)

In [25]:
all_tokens = set()

for token_set in enc_token_sets:
    # Union with the current set
    all_tokens = all_tokens.union(token_set)

token_to_index = assign_indices(all_tokens)

# Map each doc's tokens to indices
for doc, enc_tokens in zip(docs, enc_token_sets):

    cur_token_to_index = map_to_indices(enc_tokens, token_to_index)
    
    # Return the mapped tokens to doc
    doc.set_indices(cur_token_to_index)

## 4. One-Hot Vectors

Now let's learn how to get one-hot vectors from documents present on multiple workers. 

In [26]:
vocab_size = len(all_tokens)  # vocabulary size

## 5. Diffference between our approach and PSI


Extensive research work already present in the field of PSI can be referred, but our problem statement is slightly different from PSI
- In PSI, the output (set intersection) is returned to one of the parties. In our case, it would be better if **we don't let any worker know the set intersection**. Thus avoiding leaking any info to the workers about the data present on the other workers. They just receive their tokens with indices assigned to each token. Nothing else.